# Document Overview
**Purpose:**
1. Generate training and testing sequences: 
2. Generate negative samples for each sequence
3. Generate category distribution matrix for each collective POI
4. Generate index map for reindexed POIs and Users

**Input file:** 
1. Original Data: 'data_CAL.csv'
2. Helper Function Libaray: 'Helper_Functions.py'

**Output file:** 
1. Sample sets consisting POI, distance, time, type, category, negative sequence
2. POI, user, category id mapping from old to new
3. Collective POI's category distribution dictionary
4. POI distance matrix 
5. Other paramters: POI max_distance and max_sequence_length

**Creation Date:** 4th Nov 2019

In [1]:
# dependencies
import numpy as np
import pandas as pd
import os
# import argparse

In [2]:
# import helper functions

import Helper_Functions as Helper

### Adjustable parameters:

1. **small_sample** *(boolean)*: Whether to use a small sample (1000 visits) for testing
2. **augment_sample** *(boolean)*: Whether to perform sample augmentation
3. **pad_data** *(boolean)*: Whether to perform padding on data sequence

4. **min_seq_len** *(int)*: Minimum No. POIs for a valid sequence
5. **min_seq_num** *(int)*: Minimun No. valid sequences for a valid user
6. **neg_sample_num** *(int)*: Number of negative samples for each POI

In [3]:
# setup parameters (for ipython execution)

small_sample = False
augment_sample = True
pad_data = False

min_seq_len = 2
min_seq_num = 2
neg_sample_num = 5

## 1.Import data

In [4]:
if small_sample:  
    data = pd.read_csv('./data_PHO.csv')[:20000] 
else: 
    data = pd.read_csv('./data_PHO.csv')

In [5]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Location_id,POI_id,POI_Type,org_id,User_id,TimeStamp,Zone,Latitude,Longitude,Category,Yelp_id,name,stars,Time,Date,Local_sg_time,L2_id
0,18811,18811,1275,1275,Independent,51780d46e4b0d1b09374318d,8,Mon Jun 03 15:03:41 +0000 2013,-420,33.434071,-111.994311,Bakery,ux6FCssPmLx5LrWzTHvxWg,Paradise Bakery & Cafe,3.0,15:03:41,2013603,03/06/2013 15:03,78
1,18812,18812,1275,1275,Independent,51780d46e4b0d1b09374318d,8,Mon Jun 03 15:30:16 +0000 2013,-420,33.434071,-111.994311,Bakery,ux6FCssPmLx5LrWzTHvxWg,Paradise Bakery & Cafe,3.0,15:30:16,2013603,03/06/2013 15:30,78
2,18813,18813,1283,1283,Independent,516f3ce7e4b0ea41c9486d57,8,Mon Jun 03 15:31:32 +0000 2013,-420,33.434756,-111.994200,Mexican Restaurant,R6cWq0FkVfV_Ee-YttlPyA,Los Taquitos,2.0,15:31:32,2013603,03/06/2013 15:31,6
3,11532,11532,2204,2204,Independent,4d0c560d3bbaa143b79a35da,55,Mon Jan 21 14:12:59 +0000 2013,-420,33.432530,-112.043025,Rental Car Location,n0SSApg8pIMu9uWr7IvxcQ,Fox Rent A Car,1.5,14:12:59,2013121,21/01/2013 14:12,38
4,11533,11533,2198,2198,Independent,50789ab1e4b0eb8b832d778e,55,Mon Jan 21 14:13:58 +0000 2013,-420,33.427178,-112.035327,Hobby Shop,zdM9p4ZOXUb-kdRr8VPGQg,K1 Speed,3.0,14:13:58,2013121,21/01/2013 14:13,65


In [6]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Location_id', 'POI_id', 'POI_Type',
       'org_id', 'User_id', 'TimeStamp', 'Zone', 'Latitude', 'Longitude',
       'Category', 'Yelp_id', 'name', 'stars', 'Time', 'Date', 'Local_sg_time',
       'L2_id'],
      dtype='object')

## 2. Generate Visit Sequence 
Generate valid index sequences for each valid user

In [7]:
# form visit sequences 

visit_sequences, max_seq_len, valid_visits, user_reIndex_mapping = Helper.generate_sequence(data, min_seq_len, min_seq_num)

assert bool(visit_sequences), 'no qualified sequence after filtering!' # check if output sequence is empty

[========================================================================] 100%


In [8]:
Helper.peep_dictionary(visit_sequences)

168  :
 [list([73, 74]) list([22, 25]) list([27, 28]) list([33, 34, 35])
 list([53, 54]) list([59, 60]) list([63, 64]) list([134, 135])
 list([108, 109]) list([49, 50, 51]) list([159, 160])
 list([143, 144, 145]) list([149, 150]) list([180, 181]) list([152, 153])]
dictionary size:  104


In [9]:
max_seq_len 

12

In [10]:
len(valid_visits)

1351

In [11]:
user_reIndex_mapping

array([   168,    233,    257,    445,    598,   1228,   1816,   2706,
         3975,   4134,   4834,   4846,   5303,   5846,   7031,   7821,
        11185,  11800,  12164,  12310,  13301,  13815,  16473,  17128,
        18054,  18482,  18884,  19831,  21155,  22191,  22431,  24268,
        26121,  26253,  26632,  28379,  30936,  33814,  34621,  35443,
        35787,  36623,  37357,  37691,  37945,  39360,  43232,  43534,
        45067,  46745,  46785,  47143,  48796,  53457,  53849,  54383,
        54867,  55563,  55751,  58384,  59925,  60770,  62865,  64249,
        64723,  65342,  65794,  69249,  71705,  72766,  79898,  81464,
        85968,  86822,  91365,  92745,  94838,  98617, 111120, 114527,
       120997, 132692, 135315, 136457, 136686, 136755, 142862, 143450,
       150383, 156678, 157710, 157740, 164596, 164745, 165834, 170928,
       172564, 202121, 205655, 206461, 229046, 231504, 235179, 256921],
      dtype=int64)

In [12]:
# augment sequences (optional)

if augment_sample:
#     visit_sequences = Helper.aug_sequence(visit_sequences, min_len=3)
    visit_sequences, ground_truth_dict = Helper.aug_sequence(visit_sequences, min_len=3)

In [13]:
Helper.peep_dictionary(visit_sequences)

168  :
 [list([73, 74]) list([22, 25]) list([27, 28]) list([33, 34, 35])
 list([53, 54]) list([59, 60]) list([63, 64]) list([134, 135])
 list([108, 109]) list([49, 50, 51]) list([159, 160])
 list([143, 144, 145]) list([149, 150]) list([180, 181]) list([152, 153])]
dictionary size:  104


In [14]:
Helper.peep_dictionary(ground_truth_dict)

168  :
 [[ 74]
 [ 25]
 [ 28]
 [ 35]
 [ 54]
 [ 60]
 [ 64]
 [135]
 [109]
 [ 51]
 [160]
 [145]
 [150]
 [181]
 [153]]
dictionary size:  104


In [15]:
# pad sequences (optional)

if pad_data:
    
    visit_sequences = Helper.pad_sequence(visit_sequences, max_seq_len)

In [16]:
Helper.peep_dictionary(visit_sequences)

168  :
 [list([73, 74]) list([22, 25]) list([27, 28]) list([33, 34, 35])
 list([53, 54]) list([59, 60]) list([63, 64]) list([134, 135])
 list([108, 109]) list([49, 50, 51]) list([159, 160])
 list([143, 144, 145]) list([149, 150]) list([180, 181]) list([152, 153])]
dictionary size:  104


## 3. Prepare Input Sequences
Five input sequences paralleled with the Visit Sequence are prepared:
1. POI sequence
2. Distance sequence
3. Time sequence
4. Type sequence
5. Category sequence

In [17]:
# generate POI sequence

POI_sequences, POI_reIndex_mapping = Helper.generate_POI_sequences(data, visit_sequences)

In [18]:
POI_sequences[0] # POI_sequence for first user

[[559, 251],
 [587, 585],
 [591, 132],
 [118, 127, 157],
 [159, 157],
 [412, 145],
 [591, 157],
 [74, 72],
 [272, 271],
 [106, 131, 58],
 [93, 95],
 [591, 176, 72],
 [127, 591],
 [127, 585],
 [131, 168]]

In [19]:
POI_reIndex_mapping

array([  12,   14,   38,   41,   56,   59,   67,   87,   88,   89,   90,
         92,   94,   96,  100,  106,  182,  187,  202,  205,  208,  213,
        215,  232,  243,  244,  253,  257,  263,  265,  268,  280,  291,
        293,  297,  300,  303,  314,  325,  331,  334,  339,  346,  348,
        354,  357,  359,  361,  364,  371,  377,  382,  383,  387,  401,
        402,  403,  406,  418,  424,  427,  432,  433,  436,  437,  438,
        443,  446,  447,  448,  453,  454,  457,  459,  460,  462,  463,
        464,  470,  471,  473,  474,  477,  478,  479,  482,  483,  484,
        486,  495,  496,  498,  499,  501,  504,  506,  507,  508,  511,
        512,  515,  516,  518,  521,  522,  527,  529,  530,  531,  532,
        533,  537,  545,  546,  549,  551,  563,  565,  587,  589,  591,
        594,  595,  596,  601,  605,  606,  607,  608,  614,  618,  621,
        623,  626,  627,  628,  629,  641,  653,  654,  657,  658,  662,
        663,  664,  671,  674,  680,  684,  692,  6

In [20]:
# generate distance sequence

dist_sequences, max_dist = Helper.generate_dist_sequences(data, visit_sequences)

In [21]:
dist_sequences[0] # dist_sequence for first user # can perform analysis

[[0, 7],
 [0, 7],
 [0, 5],
 [0, 1, 3],
 [0, 2],
 [0, 12],
 [0, 3],
 [0, 1],
 [0, 1],
 [0, 3, 3],
 [0, 1],
 [0, 5, 5],
 [0, 4],
 [0, 5],
 [0, 5]]

In [22]:
max_dist # maximum distance between two consecutive visits 

36

In [23]:
# generate time sequence

time_sequences = Helper.generate_time_sequences(data, visit_sequences)

In [24]:
time_sequences[0] # time_sequence for first user

[[0, 13],
 [0, 1],
 [0, 4],
 [2, 3, 18],
 [1, 21],
 [1, 6],
 [21, 22],
 [3, 5],
 [2, 4],
 [3, 21, 22],
 [18, 18],
 [0, 0, 1],
 [0, 1],
 [3, 3],
 [4, 20]]

In [25]:
# generage Type sequence

type_sequences = Helper.generate_type_sequence(data, visit_sequences)

In [26]:
type_sequences[0] # type_sequence for first user

[[1, 0],
 [1, 1],
 [1, 0],
 [0, 0, 0],
 [0, 0],
 [0, 0],
 [1, 0],
 [0, 0],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [1, 0, 0],
 [0, 1],
 [0, 1],
 [0, 0]]

In [27]:
# generate category sequence

cat_sequences, cat_reIndex_mapping = Helper.generate_cat_sequences(data, visit_sequences)

In [28]:
cat_sequences[0] # cat_sequence for first user

[[84, 108],
 [25, 21],
 [111, 0],
 [2, 24, 3],
 [42, 3],
 [83, 16],
 [47, 3],
 [126, 3],
 [37, 37],
 [104, 8, 52],
 [24, 108],
 [47, 8, 3],
 [24, 111],
 [24, 22],
 [8, 24]]

In [29]:
cat_reIndex_mapping 

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        29,  30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,
        44,  45,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  61,  62,  63,  64,  65,  67,  68,  70,  71,  72,  73,
        74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
        87,  88,  91,  92,  93,  94,  95,  96,  98,  99, 100, 101, 102,
       103, 104, 106, 108, 110, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 128, 129, 130, 132, 133, 136, 139, 140, 141, 142,
       143, 144, 145, 147, 150, 152, 153, 154, 156, 159, 163, 164, 165,
       166, 168, 172, 176, 180, 184, 185, 190, 194, 196, 197, 198, 201,
       206, 212, 220, 222, 229, 230, 235, 243, 245, 249], dtype=int64)

In [30]:
# generate ground truth for each sequence

ground_truth_sequences = Helper.generate_ground_truth_sequences(data, ground_truth_dict, POI_reIndex_mapping)

In [31]:
ground_truth_sequences[0]

[[251],
 [585],
 [132],
 [157],
 [157],
 [145],
 [157],
 [72],
 [271],
 [58],
 [95],
 [72],
 [591],
 [585],
 [168]]

In [32]:
# generate specific poi ground truth for each sequence

specific_poi_sequences = Helper.generate_specific_poi_sequences(data, ground_truth_dict)

In [33]:
specific_poi_sequences[0]

[[1293],
 [855],
 [623],
 [707],
 [707],
 [671],
 [707],
 [457],
 [1495],
 [418],
 [506],
 [457],
 [2717],
 [867],
 [751]]

## 4. Extra Data Preperation

### Collective POI's category distribution

For each collective POI, count the number stores belongs to each category it has.
The distribution is recorded in a 2-layer dictionary of form:

{ POI_id (new id) : { category_id (new id): store count (int)} }

In [34]:
# generate collective POI's category distribution

poi_cat_distrib = Helper.generate_cat_distrib(data, valid_visits, POI_reIndex_mapping, cat_reIndex_mapping)

C:\Users\wuziq\Desktop\Model Implementation\Helper_Functions.py:711: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  collective_POI_visit['L2_id'] = collective_POI_visit['L2_id'].apply(lambda x: _old_id_to_new(cat_reIndex_mapping, x))


In [35]:
Helper.peep_dictionary(poi_cat_distrib)

587  :
 Counter({15: 2, 25: 1, 12: 1, 54: 1, 84: 1})
dictionary size:  51


In [36]:
poi_cat_distrib

{587: Counter({25: 1, 15: 2, 12: 1, 54: 1, 84: 1}),
 585: Counter({21: 11,
          22: 6,
          11: 10,
          76: 4,
          117: 1,
          8: 2,
          31: 1,
          68: 1,
          15: 1}),
 591: Counter({111: 5,
          47: 3,
          4: 5,
          29: 1,
          49: 1,
          5: 3,
          1: 3,
          54: 1,
          106: 1,
          0: 2,
          14: 6,
          32: 3,
          8: 5,
          25: 1,
          33: 1,
          135: 1,
          7: 1}),
 559: Counter({84: 2, 128: 3, 1: 4, 42: 1}),
 584: Counter({26: 11, 86: 3, 104: 2, 15: 1, 42: 2, 3: 1}),
 556: Counter({26: 4, 86: 5, 29: 1}),
 551: Counter({1: 2, 7: 2, 105: 1, 15: 1, 8: 2, 6: 1}),
 577: Counter({20: 3, 72: 3, 22: 4, 69: 2, 32: 4, 3: 1, 11: 1}),
 594: Counter({88: 1, 35: 3, 22: 1, 108: 1, 3: 1}),
 554: Counter({28: 3, 52: 1, 69: 1, 33: 1}),
 592: Counter({15: 1, 18: 1, 16: 2, 11: 2, 73: 1}),
 573: Counter({75: 1, 1: 1, 72: 1, 11: 1}),
 544: Counter({74: 1, 16: 1}),
 569:

In [37]:
valid_visit_data = data[data.index.isin(valid_visits)]

### Negative Samples for Each Sequence

For each user's each sequence, generate 'neg_sample_num' number of negative POIs

Negative POIs statisfy following criteria:

1. The POI does not appear in the true sequence 

2. The distance between:
    *a) negative POI and true destination* and 
    *b) true second last POI and true destination*
   should be as close as possible
   
The output neg_sequences should be a 3d array of shape [user, seq, neg_sample]

In [38]:
# store distance between each valid POI (time consuming)
    
dist_mat = Helper.generate_POI_dist_mat(data, POI_reIndex_mapping)

[========================================================================] 100%


In [39]:
dist_mat

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.94082347,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.47669455,  1.19565425,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [20.40955868, 19.84621041, 18.65182092, ...,  0.        ,
         0.        ,  0.        ],
       [19.55633596, 19.10232811, 17.90684612, ...,  1.4017442 ,
         0.        ,  0.        ],
       [30.81347388, 30.23802813, 29.04517385, ..., 10.40641281,
        11.31506108,  0.        ]])

In [40]:
# generate negative samples 

neg_sequences = Helper.generate_neg_sequences(POI_sequences, dist_mat, neg_sample_num, data, POI_reIndex_mapping, cat_reIndex_mapping)

In [41]:
neg_sequences[0] # negative samples for each sequence of 1st user

[[[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]],
 [[594, -1, 1], [592, -1, 1], [591, -1, 1], [590, -1, 1], [589, -1, 1]],
 [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]],
 [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]],
 [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]],
 [[594, -1, 1], [592, -1, 1], [593, -1, 1], [464, 15, 0], [463, 6, 0]],
 [[594, -1, 1], [592, -1, 1], [591, -1, 1], [590, -1, 1], [589, -1, 1

In [42]:
cat_reIndex_mapping

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        29,  30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,
        44,  45,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  61,  62,  63,  64,  65,  67,  68,  70,  71,  72,  73,
        74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
        87,  88,  91,  92,  93,  94,  95,  96,  98,  99, 100, 101, 102,
       103, 104, 106, 108, 110, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 128, 129, 130, 132, 133, 136, 139, 140, 141, 142,
       143, 144, 145, 147, 150, 152, 153, 154, 156, 159, 163, 164, 165,
       166, 168, 172, 176, 180, 184, 185, 190, 194, 196, 197, 198, 201,
       206, 212, 220, 222, 229, 230, 235, 243, 245, 249], dtype=int64)

In [43]:
# generate poi_cat_specific_poi_dict mapping

grouped = data.groupby(['POI_id', 'L2_id'])['Location_id'].unique().apply(list)

grouped

POI_id  L2_id
0       27                                                     [0]
1       145                                                    [1]
10      8                                                     [10]
11      48                                                    [11]
12      25                                                    [12]
13      5                                                     [13]
14      141                                                   [14]
15      99                                                    [15]
16      29                                                    [16]
17      52                                                    [17]
18      25                                                    [18]
19      1                                                     [19]
20      3                                                     [20]
21      195                                                   [21]
22      147                                     

In [44]:
# generate poi_cat_specific_poi_dict

poi_cat_specific_poi_dict = {}

prev_poi = grouped.index[0][0]

cat_specific_poi_dict = {}

cat_specific_poi_dict[grouped.index[0][1]] = grouped[grouped.index[0]]

for index in grouped.index:

    if index[0] not in poi_cat_specific_poi_dict.keys():  
        
        poi_cat_specific_poi_dict[prev_poi] = cat_specific_poi_dict
        
        cat_specific_poi_dict = {}
        
        prev_poi = index[0]
        
        poi_cat_specific_poi_dict[index[0]] = {}
        
    cat_specific_poi_dict[index[1]] = grouped[index]
    
poi_cat_specific_poi_dict[prev_poi] = cat_specific_poi_dict

## 5. Form Sample Sets

Concatenate five sequences to form a sample, which is a tuple consists of: (POI_seq, dist_seq, time_seq, type_seq, cat_seq, neg_samplw)

Organise samples according to users in a dictionary of form:

{ User_id (new id) : sample sets } 

In [45]:
# form sample set for each user

sample_sets = Helper.form_sample_sets(POI_sequences, dist_sequences, time_sequences, type_sequences, cat_sequences, ground_truth_sequences, specific_poi_sequences, neg_sequences)

Total user: 104 -- Total sample: 647


In [46]:
Helper.peep_dictionary(sample_sets)

0  :
 [([559, 251], [0, 7], [0, 13], [1, 0], [84, 108], [251], [1293], [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]), ([587, 585], [0, 7], [0, 1], [1, 1], [25, 21], [585], [855], [[594, -1, 1], [592, -1, 1], [591, -1, 1], [590, -1, 1], [589, -1, 1]]), ([591, 132], [0, 5], [0, 4], [1, 0], [111, 0], [132], [623], [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]), ([118, 127, 157], [0, 1, 3], [2, 3, 18], [0, 0, 0], [2, 24, 3], [157], [707], [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]]), ([159, 157], [0, 2], [1, 21], [0, 0], [42, 3], [157], [707], [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]]), ([412, 145], [0, 12], [1, 6], [0, 0], [83, 16], [145], [671], [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]), ([591, 157], [0, 3], [21, 22], [1, 0], [47, 3], [157], [707], [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]), ([74, 72], [0, 1], [3, 5], [0, 0], [126, 

In [47]:
sample_sets

{0: [([559, 251],
   [0, 7],
   [0, 13],
   [1, 0],
   [84, 108],
   [251],
   [1293],
   [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]),
  ([587, 585],
   [0, 7],
   [0, 1],
   [1, 1],
   [25, 21],
   [585],
   [855],
   [[594, -1, 1], [592, -1, 1], [591, -1, 1], [590, -1, 1], [589, -1, 1]]),
  ([591, 132],
   [0, 5],
   [0, 4],
   [1, 0],
   [111, 0],
   [132],
   [623],
   [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]),
  ([118, 127, 157],
   [0, 1, 3],
   [2, 3, 18],
   [0, 0, 0],
   [2, 24, 3],
   [157],
   [707],
   [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]]),
  ([159, 157],
   [0, 2],
   [1, 21],
   [0, 0],
   [42, 3],
   [157],
   [707],
   [[297, 29, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0], [396, 3, 0]]),
  ([412, 145],
   [0, 12],
   [1, 6],
   [0, 0],
   [83, 16],
   [145],
   [671],
   [[297, 29, 0], [392, 34, 0], [393, 34, 0], [394, 34, 0], [395, 6, 0]]),
  ([591, 157],
   [0, 3],
   [21, 22],
  

# 6. Output Files

In [48]:
# set output directory

dir = './np_save_PHO/'
if small_sample:
    dir = './test_np_save_PHO/'

In [49]:
# create directory if not exists

if not os.path.exists(dir):
    os.makedirs(dir)

In [50]:
# save concatenated samples

Helper.save_dict(sample_sets, dir + 'sample_sets.pkl')

In [51]:
# save id mappings

np.save(dir + 'POI_reIndex_mapping.npy', POI_reIndex_mapping)
np.save(dir + 'user_reIndex_mapping.npy', user_reIndex_mapping)
np.save(dir + 'cat_reIndex_mapping.npy', cat_reIndex_mapping)

In [52]:
# save collective POI's category distribution dictionary

Helper.save_dict(poi_cat_distrib, dir + 'poi_cat_distrib.pkl')
Helper.save_dict(poi_cat_specific_poi_dict, dir + 'poi_cat_specific_poi_dict.pkl')

In [53]:
# save POI distance matrix 

np.save(dir + 'dist_mat.npy', dist_mat)

In [54]:
# save other relavant parameters

np.save(dir + 'max_dist.npy', max_dist) # max distance (for distance embedding)
np.save(dir + 'max_seq_len.npy', max_seq_len) # max sequence length (for input size)
np.save(dir + 'neg_sample_num.npy', neg_sample_num) # number of negative samples (for negative input size)